### 1. Create an IAM role that has read access to S3
[1.1 Create an IAM Role](https://learn.udacity.com/nanodegrees/nd027/parts/cd12440/lessons/2a40895d-1b82-41fa-8913-0dbffb0a4095/concepts/113f3fc2-feda-43f6-822d-8216c9336073)
<br>[1.2 Create Security Group](https://learn.udacity.com/nanodegrees/nd027/parts/cd12440/lessons/2a40895d-1b82-41fa-8913-0dbffb0a4095/concepts/42d13cf6-3983-443d-b389-47cf27cee96b)
<br>[1.3 Create an IAM User](https://learn.udacity.com/nanodegrees/nd027/parts/cd12440/lessons/2a40895d-1b82-41fa-8913-0dbffb0a4095/concepts/1b3e6b85-d159-4217-aa2c-408a29a08fb5)

### 2. Edit the file `dwh.cfg`
[AWS]
<br>KEY=\<YOUR_AWS_KEY>
<br>SECRET=\<YOUR_AWS_SECRET>
    
<br>[DWH]
<br>DWH_CLUSTER_TYPE=multi-node
<br>DWH_NUM_NODES=4
<br>DWH_NUM_TYPE=dc2.large

<br>DWH_IAM_ROLE_NAME=dwhRole
<br>DWH_CLUSTER_IDENTIFIER=dwhCluster
<br>DWH_DB=dwh
<br>DWH_DB_USER=dwhuser
<br>DWH_DB_PASSWORD=Passw0rd
<br>DWH_PORT=5439


### 3. Launch a Redshift Cluster

In [ ]:
#!pip install boto3

In [ ]:
import pandas as pd
import boto3
import json

In [ ]:
# load DWH Params from a file
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

In [ ]:
# create clients for EC2, S3, IAM, and Redshift
import boto3

ec2 = boto3.resource('ec2',
                     region_name = 'us-east-1',
                     aws_access_key_id = KEY,
                     aws_secret_access_key = SECRET
                    )

s3 = boto3.resource('s3',
                     region_name = 'us-east-1',
                     aws_access_key_id = KEY,
                     aws_secret_access_key = SECRET
                    )

iam = boto3.client('iam',
                        region_name = 'us-east-1',
                        aws_access_key_id = KEY,
                        aws_secret_access_key = SECRET)

redshift = boto3.client('redshift',
                        region_name = 'us-east-1',
                        aws_access_key_id = KEY,
                        aws_secret_access_key = SECRET)

In [ ]:
# before running the following code
# give IAM user `AdministratorAccess` from `Attach policies`
from botocore.exceptions import ClientError

# 1.1 Create the role,
try:
    print("1.1 Creating a new IAM Role")
    dwhRole = iam.create_role(
        Path = '/',
        RoleName = DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
    AssumeRolePolicyDocument = json.dumps(
        {'Statement': [{'Action': 'sts:AssumeRole',
                        'Effect': 'Allow',
                        'Principal': {'Service': 'redshift.amazonaws.com'}}],
         'Version': '2012-10-17'})
    )
except Exception as e:
    print(e)

print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName = DWH_IAM_ROLE_NAME,
                       PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName = DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

In [ ]:
try:
    response = redshift.create_cluster(
        #HW
        ClusterType = DWH_CLUSTER_TYPE,
        NodeType = DWH_NODE_TYPE,
        NumberOfNodes = int(DWH_NUM_NODES),
        
        #Identifiers & Credentials
        DBName = DWH_DB,
        ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
        MasterUsername = DWH_DB_USER,
        MasterUserPassword = DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles = [roleArn]
    )
except Exception as e:
    print(e)

In [ ]:
# run this block several times until the cluster status becomes `Available`
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", \
                  "NumberOfNodes", "VpcId"]
    x = [(k, v) for k, v in props.items() if k in keysToShow]
    return pd.DataFrame(data = x, columns = ["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier = DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
# DO NOT RUN THIS unless the cluster status becomes "Available"
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

### 4. Add Redshift Database and IAM Role into `dwh.cfg`
<br>[CLUSTER]
<br>HOST=\<YOUR HOST>
<br>DB_NAME=dwh
<br>DB_USER=dwhuser
<br>DB_PASSWORD=Passw0rd
<br>DB_PORT=5439
<br>REGION=us-west-2

<br>[IAM_ROLE]
<br>ARN=\<YOUR ARN>

<br>[S3]
<br>LOG_DATA='s3://udacity-dend/log_data'
<br>SONG_DATA='s3://udacity-dend/song_data'
<br>LOG_JSONPATH='s3://udacity-dend/log_json_path.json'

In [ ]:
try:
    vpc = ec2.Vpc(id = myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName = defaultSg.group_name,
        CidrIp = '0.0.0.0/0',
        IpProtocol = 'TCP',
        FromPort = int(DWH_PORT),
        ToPort = int(DWH_PORT)
    )
except Exception as e:
    print(e)

In [ ]:
%load_ext sql

In [ ]:
conn_string = 'postgresql://{}:{}@{}:{}/{}'.format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
print(conn_string)
%sql $conn_string

### 5. Drop and Create Tables
```
% python create_tables.py
```
### 6. Run `etl.py`
```
% python etl.py
```

### 7. Test Example Queries
**7.1 Data Rows Check**

In [ ]:
%sql SELECT COUNT(*) FROM staging_events

In [ ]:
%sql SELECT COUNT(*) FROM staging_songs

In [ ]:
%sql SELECT COUNT(*) FROM songplays

In [ ]:
%sql SELECT COUNT(*) FROM users

In [ ]:
%sql SELECT COUNT(*) FROM songs

In [ ]:
%sql SELECT COUNT(*) FROM artists

In [ ]:
%sql SELECT COUNT(*) FROM time

**7.2 Song Play Analysis**
1. Which 20 songs are most popular?

In [ ]:
%%time
%%sql
SELECT s.title,
       COUNT(*) times
FROM songs s
JOIN songplays sp
ON s.song_id = sp.song_id
GROUP BY s.title
ORDER BY times DESC
LIMIT 20;

2. Which 10 artists are most popular?

In [ ]:
%%time
%%sql
SELECT a.name,
       COUNT(*) hotness
FROM songplays sp
LEFT JOIN artists a
ON sp.artist_id = a.artist_id
GROUP BY a.name
ORDER BY hotness DESC
LIMIT 10

3. What time has the highest traffic on the app?

In [ ]:
%%time
%%sql
SELECT t.hour,
       COUNT(*) hotness
FROM songplays sp
LEFT JOIN time t
ON t.start_time = sp.start_time
GROUP BY t.hour
ORDER BY hotness DESC
LIMIT 5

4. Who are the 50 most active users?

In [ ]:
%%time
%%sql
SELECT u.user_id,
       u.first_name,
       u.last_name,
       u.gender,
       u.level,
       COUNT(*) activeness
FROM songplays sp
LEFT JOIN users u
ON sp.user_id = u.user_id
GROUP BY u.user_id
ORDER BY activeness DESC
LIMIT 50

### 8. Delete the Redshift Cluster

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

In [ ]:
# run this block several times until the cluster really deleted
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!